# Common examples

## Homogeneous dataset

In [1]:
import copy
import pickle

import numpy as np
from datasets import load_dataset
from plaid.bridges.huggingface_bridge import (
    huggingface_dataset_to_plaid,
    huggingface_description_to_problem_definition,
)
from plaid.containers.sample import Sample
from torch.utils.data import DataLoader

from plaid_bridges.common import HomogeneousBridge

/home/fabien/miniconda3/envs/plaid-bridged/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
hf_dataset = load_dataset("PLAID-datasets/VKI-LS59", split="all_samples[:10]")
sample = Sample.model_validate(pickle.loads(hf_dataset[0]["sample"]))

pb_def = huggingface_description_to_problem_definition(hf_dataset.info.description)
ids = pb_def.get_split("train")[:10]

dataset, _ = huggingface_dataset_to_plaid(hf_dataset, ids=ids, processes_number=5)
print(dataset)

all_feat_ids = dataset[0].get_all_features_identifiers()

Converting Hugging Face dataset to plaid dataset...


100%|██████████| 10/10 [00:00<00:00, 51.56it/s]


Dataset(10 samples, 8 scalars, 0 time_series, 8 fields)


### Field inputs and scalar outputs

In [3]:
scalar_features = [f for f in all_feat_ids if "scalar" in f.values()]
field_features = [
    f for f in all_feat_ids if "field" in f.values() and "Base_2_2" in f.values()
]

in_features_identifiers = field_features
out_features_identifiers = scalar_features

print(in_features_identifiers)
print(out_features_identifiers)

[{'type': 'field', 'name': 'sdf', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': 0.0}, {'type': 'field', 'name': 'rov', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': 0.0}, {'type': 'field', 'name': 'ro', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': 0.0}, {'type': 'field', 'name': 'rou', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': 0.0}, {'type': 'field', 'name': 'roe', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': 0.0}, {'type': 'field', 'name': 'mach', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': 0.0}, {'type': 'field', 'name': 'nut', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': 0.0}]
[{'type': 'scalar', 'name': 'Pr'}, {'type': 'scalar', 'name': 'Q'}, {'type': 'scalar', 'name': 'Tr'}, {'type': 'scalar', 'name': 'angle_in'}, {'type': 'scalar', 'name': 'angle_out'}, {'type': 's

In [4]:
bridge = HomogeneousBridge()
homogen_dataset = bridge.convert(
    dataset, in_features_identifiers, out_features_identifiers
)

loader = DataLoader(
    homogen_dataset,
    batch_size=2,
    shuffle=False,
)

out_feat_id = out_features_identifiers[0]
before = copy.deepcopy(dataset[ids[1]].get_feature_from_identifier(out_feat_id))

predictions = []
for batch_x, batch_y in loader:
    for torch_sample in batch_y:
        predictions.append(torch_sample.detach().cpu().numpy())

dataset_pred = bridge.restore(dataset, predictions, out_features_identifiers)

after = copy.deepcopy(dataset_pred[ids[1]].get_feature_from_identifier(out_feat_id))

print("Error after transform then inverse transform (2nd sample):")
print(np.linalg.norm(after - before) / np.linalg.norm(before))

Error after transform then inverse transform (2nd sample):
0.0


### Scalar inputs and Field outputs

In [5]:
in_features_identifiers = scalar_features
out_features_identifiers = field_features

print(in_features_identifiers)
print(out_features_identifiers)

[{'type': 'scalar', 'name': 'Pr'}, {'type': 'scalar', 'name': 'Q'}, {'type': 'scalar', 'name': 'Tr'}, {'type': 'scalar', 'name': 'angle_in'}, {'type': 'scalar', 'name': 'angle_out'}, {'type': 'scalar', 'name': 'eth_is'}, {'type': 'scalar', 'name': 'mach_out'}, {'type': 'scalar', 'name': 'power'}]
[{'type': 'field', 'name': 'sdf', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': 0.0}, {'type': 'field', 'name': 'rov', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': 0.0}, {'type': 'field', 'name': 'ro', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': 0.0}, {'type': 'field', 'name': 'rou', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': 0.0}, {'type': 'field', 'name': 'roe', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': 0.0}, {'type': 'field', 'name': 'mach', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': 0.0}, {'type': 'f

In [6]:
bridge = HomogeneousBridge()
homogen_dataset = bridge.convert(
    dataset, in_features_identifiers, out_features_identifiers
)

loader = DataLoader(
    homogen_dataset,
    batch_size=2,
    shuffle=False,
)

out_feat_id = out_features_identifiers[0]
before = copy.deepcopy(dataset[ids[1]].get_feature_from_identifier(out_feat_id))

predictions = []
for batch_x, batch_y in loader:
    for torch_sample in batch_y:
        predictions.append(torch_sample.detach().cpu().numpy())

dataset_pred = bridge.restore(dataset, predictions, out_features_identifiers)

after = copy.deepcopy(dataset_pred[ids[1]].get_feature_from_identifier(out_feat_id))

print("Error after transform then inverse transform (2nd sample):")
print(np.linalg.norm(after - before) / np.linalg.norm(before))

Error after transform then inverse transform (2nd sample):
0.0
